## Visualizing Bicycle Counts in Germany

In [72]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime
from datetime import date
from dssg_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Counting Stations

Metadata about each counting station is available.

In [21]:
df_meta = pd.read_csv("../data/fetched_eco_bike_data.csv")
df_meta.rename(columns={'number': 'station_code'}, inplace=True)
df_meta.set_index(['city', 'station_code'], inplace=True)
display(df_meta)
len(df_meta)

latitude  longitude    id_list  \
city          station_code                                    
Aschaffenburg 1             49.973150   9.150790  100046798   
Berlin        1             52.514066  13.417751  100024661   
              2             52.527180  13.372020  100032152   
              3             52.513739  13.474367  100032154   
              4             52.566810  13.412170  100032155   
...                               ...        ...        ...   
Stuttgart     13            48.826000   9.214880  100063205   
              14            48.780270   9.177410  100063206   
              15            48.769761   9.172275  100063225   
Osnabrück     1             52.279965   8.055056  100050631   
Ludwigsburg   1             48.893189   9.192943  100041999   

                            Distance to center [km] fetched_url  
city          station_code                                       
Aschaffenburg 1                              0.1432      #NAME?  
Berlin        1                              1.9827      #NAME?  
              2                              1.6032      #NAME?  
              3                              5.7977      #NAME?  
              4                              5.7547      #NAME?  
...                                             ...         ...  
Stuttgart     13                             5.8718      #NAME?  
              14                             0.2782      #NAME?  
              15                             1.1201      #NAME?  
Osnabrück     1                              1.0234      #NAME?  
Ludwigsburg   1                              0.3506      #NAME?  

[85 rows x 5 columns]

85

We can show the location of each counting station on a map.

In [22]:
m = folium.Map(location=[52.514066, 13.417751])
for i, coordinates in enumerate(list(zip(df_meta.latitude, df_meta.longitude))):
    key = df_meta.index[i][0] + str(df_meta.index[i][1])
    folium.Marker(coordinates, popup=key, tooltip="Click me", icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{key}</div>""")
    ).add_to(m)
m

### Bike Accidents

We can load the accident data to visualize it as a heatmap.

In [23]:
df_accidents = pd.read_csv("../data/accidents/cleaned.csv")
display(df_accidents)

,UJAHR,UMONAT,UWOCHENTAG,USTUNDE,latlon,size,longitude,latitude
0,2018,1,1,13,"(13.253224995887086, 52.445928149352355)",1,13.253225,52.445928
1,2018,1,1,14,"(13.41193910223544, 52.531282229269756)",1,13.411939,52.531282
2,2018,1,1,17,"(13.455604233979928, 52.43610101156052)",1,13.455604,52.436101
3,2018,1,1,17,"(13.478171429150134, 52.514969983178915)",1,13.478171,52.514970
4,2018,1,1,18,"(13.325952614976552, 52.49443342932684)",1,13.325953,52.494433
...,...,...,...,...,...,...,...,...
5187,2018,12,7,19,"(13.437086752163351, 52.474497557791786)",1,13.437087,52.474498
5188,2018,12,7,19,"(13.438986252262747, 52.541081188661465)",1,13.438986,52.541081
5189,2018,12,7,20,"(13.453524616321932, 52.51041762186408)",1,13.453525,52.510418
5190,2018,12,7,20,"(13.492419969190417, 52.4913399864065)",1,13.492420,52.491340


In [24]:
from folium import plugins
lat_lon = list(zip(df_accidents.latitude, df_accidents.longitude))
m.add_children(plugins.HeatMap(lat_lon, radius=15))
m

/tmp/ipykernel_222521/247102128.py:3: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  m.add_children(plugins.HeatMap(lat_lon, radius=15))


### Dynamic Counts Map

Load the combined data table and merge it with the metadata.

In [25]:
df = pd.read_csv("../data/combined.csv")
df['date'] = pd.to_datetime(df.date)
df.set_index(['station_code','city'], inplace=True)
display(df)

date  count      timestamp
station_code city                                                   
1            Aschaffenburg 2018-08-10 00:00:00      1  1533859200000
             Aschaffenburg 2018-08-10 00:15:00      0  1533860100000
             Aschaffenburg 2018-08-10 00:30:00      0  1533861000000
             Aschaffenburg 2018-08-10 00:45:00      0  1533861900000
             Aschaffenburg 2018-08-10 01:00:00      1  1533862800000
...                                        ...    ...            ...
15           Stuttgart     2021-09-29 22:45:00      7  1632955500000
             Stuttgart     2021-09-29 23:00:00      6  1632956400000
             Stuttgart     2021-09-29 23:15:00      6  1632957300000
             Stuttgart     2021-09-29 23:30:00      1  1632958200000
             Stuttgart     2021-09-29 23:45:00      4  1632959100000

[10895943 rows x 3 columns]

In [26]:
merged = df.join(df_meta)

Create a time-based mask to narrow done selection.

In [27]:
sub = merged.query('station_code > 3')
mask =((sub.date.dt.date > datetime.date(2020, 8, 10)) & 
       (sub.date.dt.date < datetime.date(2020, 8, 15))) 

sub = sub[mask]
display(sub)

date  count      timestamp   latitude  \
station_code city                                                          
4            Berlin 2020-08-11 00:00:00     40  1597104000000  52.566810   
             Berlin 2020-08-11 00:15:00      5  1597104900000  52.566810   
             Berlin 2020-08-11 00:30:00      6  1597105800000  52.566810   
             Berlin 2020-08-11 00:45:00      7  1597106700000  52.566810   
             Berlin 2020-08-11 01:00:00     16  1597107600000  52.566810   
...                                 ...    ...            ...        ...   
17           Berlin 2020-08-14 22:45:00      5  1597445100000  52.438523   
             Berlin 2020-08-14 23:00:00     12  1597446000000  52.438523   
             Berlin 2020-08-14 23:15:00     18  1597446900000  52.438523   
             Berlin 2020-08-14 23:30:00     12  1597447800000  52.438523   
             Berlin 2020-08-14 23:45:00      9  1597448700000  52.438523   

                     longitude      id_list  Distance to center [km]  \
station_code city                                                      
4            Berlin  13.412170  100032155.0                   5.7547   
             Berlin  13.412170  100032155.0                   5.7547   
             Berlin  13.412170  100032155.0                   5.7547   
             Berlin  13.412170  100032155.0                   5.7547   
             Berlin  13.412170  100032155.0                   5.7547   
...                        ...          ...                      ...   
17           Berlin  13.387829  100064714.0                   8.7306   
             Berlin  13.387829  100064714.0                   8.7306   
             Berlin  13.387829  100064714.0                   8.7306   
             Berlin  13.387829  100064714.0                   8.7306   
             Berlin  13.387829  100064714.0                   8.7306   

                    fetched_url  
station_code city                
4            Berlin      #NAME?  
             Berlin      #NAME?  
             Berlin      #NAME?  
             Berlin      #NAME?  
             Berlin      #NAME?  
...                         ...  
17           Berlin      #NAME?  
             Berlin      #NAME?  
             Berlin      #NAME?  
             Berlin      #NAME?  
             Berlin      #NAME?  

[14688 rows x 8 columns]

Create animated maps with the counts represented as dots.

In [74]:
for group in sub.groupby('city'):
    start_location = (group[1].iloc[0].latitude, group[1].iloc[0].longitude)
    create_map(group[1], city_name=group[0], start_location=start_location, export=True)

### Clustering Based Markers

In [19]:
m = folium.Map(location=[52.514066, 13.417751])
for i, coordinates in enumerate(list(zip(df_meta.latitude, df_meta.longitude))):
    key = df_meta.index[i][0] + str(df_meta.index[i][1])
    if key in city_cluster.keys():
        color = city_cluster[key]
        folium.Marker(coordinates, popup=key, tooltip=tooltip, icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: {color}">{key}</div>""")).add_to(m)
        
m